In [ ]:
%reset

In [1]:
# 加载所需的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

np.random.seed(1)

In [47]:
#参数
I = np.arange(10)
delta = 0.1
alpha = 0.05
mu = np.arange(1,11)/10
sigma = np.power(np.arange(1,11),0.5)
k = len(I)
#Paulson
Lambda = delta/2
a = np.log((k-1)/alpha)*1/(delta-Lambda)
#KN
n_0 = 100
eta =  (np.power(2*alpha/(k-1),-2/(n_0-1)) - 1)/2
h_squire = 2*eta*(n_0-1)
#样本方差
S_squire = pd.DataFrame(index=range(10),columns=range(10))

In [51]:
X_Initialization = pd.DataFrame()
for i in I:
    x = np.random.normal(loc=mu[i],scale=sigma[i],size=n_0)
    x = pd.DataFrame(x).T
    X_Initialization = pd.concat([X_Initialization,x])
X_Initialization.index = I

In [65]:
for i in I:
    x_i = X_Initialization.loc[i]
    for l in I:
        x_l = X_Initialization.loc[l]
        x_il = x_i-x_l
        s_il = np.var(x_il,ddof=1)
        S_squire.loc[i,l] = s_il
    s_ii = np.var(x_i,ddof=1)
    S_squire.loc[i,i] = s_ii
S_diagnoal = np.power(np.diagonal(S_squire),0.5)
X_bar = np.mean(X_Initialization,axis=1)

In [66]:
#Paulson
r = 0
X = pd.DataFrame(index=range(10))
while r <1000:
    X[r]=np.random.normal(loc=mu,scale = sigma)/S_diagnoal + X_bar*(1-1/S_diagnoal)
    r += 1


/var/folders/qj/s_39ngds10394t00jt5t0c4w0000gn/T/ipykernel_1271/278301157.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[r]=np.random.normal(loc=mu,scale = sigma)/S_diagnoal + X_bar*(1-1/S_diagnoal)
/var/folders/qj/s_39ngds10394t00jt5t0c4w0000gn/T/ipykernel_1271/278301157.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[r]=np.random.normal(loc=mu,scale = sigma)/S_diagnoal + X_bar*(1-1/S_diagnoal)
/var/folders/qj/s_39ngds10394t00jt5t0c4w0000gn/T/ipykernel_1271/278301157.py:5: PerformanceWarning: DataFrame is highly fr

In [67]:
np.mean(X,axis=1)


0    0.064214
1    0.247891
2    0.221265
3    0.297698
4    0.518771
5    0.669504
6    0.616621
7    0.721904
8    1.121907
9    0.812619
dtype: float64